In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import subprocess
from darknetpy.detector import Detector
import xml.etree.ElementTree as ET

In [3]:
annot_path = "./TrainVal/VOCdevkit/VOC2011/Annotations/"
image_path = "./TrainVal/VOCdevkit/VOC2011/JPEGImages/"
split_path = "./TrainVal/VOCdevkit/VOC2011/ImageSets/Segmentation/val.txt"

In [4]:
#Modifies path in order to allow for detector to access images
valid_in = open(split_path, "r")
valid_out = open("./valid.txt", "w")
for line in valid_in:
    valid_out.write("../TrainVal/VOCdevkit/VOC2011/JPEGImages/" + line[:-1] + ".jpg\n")
valid_in.close()
valid_out.close()

In [ ]:
#Runs shell script to install darknet files and run inference...takes a long time so the txt are submitted too
#This step is only included for reproducibility
subprocess.run(["chmod", "+x", "install-dependencies.sh"])
subprocess.run("./install-dependencies.sh",  capture_output=True, shell=True)

In [ ]:
def build_df(annot_path, image_path, split_path):    
    data = pd.DataFrame(columns=["image", "annotation", "IoU", "mAP", "Time"])
    split = open(split_path, "r")
    lines = split.readlines()
    split.close()
    jpg = []
    xml = []
    for i in lines:
        temp = i.split(" ")[0][:-1]
        jpg.append(image_path+temp+".jpg")
        xml.append(annot_path + temp + ".xml")
    data["image"] = jpg
    data["annotation"] = xml
    return data
    


In [ ]:
data = build_df(annot_path, image_path, split_path)
image = load_image(data["image"].iloc[0])
x = detect_objects(image[0], standard, output_layers)[1]

In [ ]:
def find_boxes(file):

    tree = ET.parse(file)
    root = tree.getroot()

    boxes = []

    for box in root.iter('object'):


        ymin = int(box.find("bndbox/ymin").text)
        xmin = int(box.find("bndbox/xmin").text)
        ymax = int(box.find("bndbox/ymax").text)
        xmax = int(box.find("bndbox/xmax").text)

        boxes.append((xmin, ymin, xmax, ymax))
        
    return boxes



In [ ]:
#Inputs are two tuples of the form (x, y, w, h)
def IoU(gt, pred):
    length_overlap = (gt[0]+gt[3]) - pred[0]
    width_overlap = (gt[1]+gt[3]) - pred[1]
    area_overlap = length_overlap * width_overlap
    area_intersect = (gt[2]*gt[3]) + (pred[2] * pred[3]) - area_overlap
    iou = area_overlap / area_intersect
    return iou
    

In [ ]:
#Inputs are lists of tuples of the form [("classification", x, y, w, h), ...,]
def mAP(gt, pred):
    output = {}
    for i in range(len(gt)):
        iou = IoU(gt[i][1:], pred[i][1:])
        if (iou >= 0.5 and gt[i][0] = None) or (iou >= 0.5 and gt[i][0] != pred[i][0])
            output[tuple(gt[i], pred[i])] = "FN"
        elif iou >= 0.5:
            output[tuple(gt[i], pred[i])] = "TP"
        else
            output[tuple(gt[i], pred[i])] = "FP"
    
        
    